In [1]:
import requests
import json
import shutil
from pydantic import BaseModel, Field
from typing import Literal, Optional

In [2]:


class SpamDetectorSmart(BaseModel):
    chain_of_thought: Optional[str] = Field(None, description="Presenting a consequent way of reasoning to decide about the text")
    label: Literal["spam", "not_spam"]

def display_response(response: requests.Response, terminal_width: int = None):
    """
    Function to handle the streaming of intermediate response pieces and display final results.
    """
    if terminal_width is None:
        terminal_width = shutil.get_terminal_size().columns

    full_response = ""
    current_line_length = 0  # Track current length of the output line

    # Iterate over streamed lines
    for line in response.iter_lines():
        if line:
            try:
                line_data = json.loads(line.decode('utf-8'))  # Decode bytes to string and parse JSON
                response_piece = line_data.get("response", "")
                full_response += response_piece  # Collect the full response

                # Calculate the new length after appending this piece
                new_length = current_line_length + len(response_piece) + 1  # Adding space between words

                if new_length <= terminal_width:
                    print(response_piece, end=" ")  # Append on the same line with a space
                    current_line_length = new_length  # Update the current line length
                else:
                    print()  # Move to the next line
                    print(response_piece, end=" ")  # Print on the new line
                    current_line_length = len(response_piece)  # Reset the current length for the new line

            except Exception as e:
                print(f"Failed to parse line: {line}, error: {e}")

    # After all parts are streamed, parse the final response
    try:
        # Handle cases where chain_of_thought is missing
        final_response = json.loads(full_response)
        label = final_response.get("label")
        chain_of_thought = final_response.get("chain_of_thought", "No chain of thought provided")  # Default if missing

        print("\n\nFinal classification result:")
        print(f"Label: {label}")
        print(f"Chain of Thought: {chain_of_thought}")
        
        # Now parse the final response using Pydantic
        model_response = SpamDetectorSmart(label=label, chain_of_thought=chain_of_thought)
        print("Pydantic Model Output:", model_response)

    except Exception as e:
        print("Failed to parse the reconstructed response:", e)




In [3]:
# Define the SMS to classify
SMS = "Win the newest Harry Potter and the Order of the Phoenix (Book 5) reply HARRY, answer 5 questions - chance to be the first among readers!"

# New API URL
url = "http://192.168.31.108:11434/api/generate"

# Payload for the request
payload = {
    "model": "qwen2",
    "prompt": f"Classify the following SMS as spam or not_spam. Provide reasoning for the classification.\n\n{SMS}\n\nResponse format: {{\"label\": \"spam\", \"chain_of_thought\": \"[reasoning here]\"}}",
    "max_length": 150,
    "temperature": 0.0  # Low temperature for deterministic output
}

# Headers for the request (if needed)
headers = {
    "Content-Type": "application/json"
}

# Make the API call
response = requests.post(url, json=payload, headers=headers, stream=True)

# Check the response status and handle the streaming display
if response.status_code == 200:
    display_response(response)
else:
    print(f"Error: {response.status_code} - {response.text}")


{" label ":  " spam ",  " chain _of _th ought ":  " The  message  is  trying 
 ent ice  the  recipient  by  offering  a  prize  ( the  newest  Harry 
 book )  in  exchange  for  rep lying  and  answering  questions .  Such 
 often  have  promotional  elements  and  require  action  from  the 
,  which  are  typical  characteristics  of  spam ." }  

Final classification result:
Label: spam
Chain of Thought: The message is trying to entice the recipient by offering a prize (the newest Harry Potter book) in exchange for replying and answering questions. Such messages often have promotional elements and require action from the recipient, which are typical characteristics of spam.
Pydantic Model Output: chain_of_thought='The message is trying to entice the recipient by offering a prize (the newest Harry Potter book) in exchange for replying and answering questions. Such messages often have promotional elements and require action from the recipient, which are typical characteristics of spam